<a href="https://colab.research.google.com/github/Blackman9t/Machine_Learning/blob/master/Toronto_Geodata_assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### First Let's import required libraries

In [65]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize  # transform json files to pandas dataframes
from geopy.geocoders import Nominatim # 
import numpy as np
import csv
!pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

print('All modules imported')

All modules imported


## Let's start scraping the wikipedia page

In [0]:
# The wikipedia site link
site_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Get the source code html data from the website

In [0]:
source = requests.get(site_link).text

Lets Use BeautifulSoup to parse it

In [0]:
soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())

Next let's get the table that contains the data we want to scrape

In [0]:
My_table = soup.find('table',{'class':'wikitable sortable'})

Let's view the table data

In [0]:
# Uncomment below to view table
# My_table

we can see that all the data we want are between the $<td>$ brackets, let's get the data between the td brackets

In [0]:
links = My_table.find_all('td')

In [0]:
# uncomment below to view links
# print(links)

Next let's loop through links and extract only the text elements

In [0]:
text_links = []

for link in links:
    text_links.append(link.text)
    
# uncommnet below to view text_links    
#text_links

### let's clean and process the table elements

Let's clean the links and keep only rows with Borough. Out of which we shall rename rows without Neighborhood as Boroughs

In [0]:
cleaned_links = []

while True:
    
    if len(text_links) < 3:
        break
    
    sub = text_links[:3]
    # If 'Not ' in borough then skip that row of data
    if 'Not ' in sub[1]:
        text_links = text_links[3:]
    else:
        cleaned_links.append(text_links[:3])
        
        # Let's strip off the \n at the end of each neighborhood data
        cleaned_links[-1][-1] = cleaned_links[-1][-1].strip('\n')
        
        # If the Borough is available but the Neighborhood is missing
        # make Neighborhood same as Borough
        if 'Not ' in cleaned_links[-1][-1]:
            cleaned_links[-1][-1] = cleaned_links[-1][-2]
        text_links = text_links[3:]
# Uncomment below to view cleaned_links       
#cleaned_links

lets check the length of the cleaned links

In [11]:
len(cleaned_links)

211

Next let's add the neighborhood data of each duplicate Postal Codes together to the first instance or row that contains the PostalCode

In [0]:
link = []
for i in range(len(cleaned_links)):
    x = cleaned_links[i][0]
    if x in link:
        cleaned_links[link.index(x)][-1] += ', ' + cleaned_links[i][-1]
    link.append(x)
    
# uncomment below
#cleaned_links

Next let's pass the cleaned _links to a data frame and set index to postal code so that we can easily work on it

In [0]:
df = pd.DataFrame(cleaned_links, columns=['PostalCode','Borough','Neighborhood'])
df.index= df.PostalCode

In [14]:
# Let's view the data frame
df.head()

,PostalCode,Borough,Neighborhood
PostalCode,,,
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,"Harbourfront, Regent Park"
M5A,M5A,Downtown Toronto,Regent Park
M6A,M6A,North York,"Lawrence Heights, Lawrence Manor"


Next let's Use the pandas duplicate method to drop duplicate index

In [0]:
df = df.loc[~df.index.duplicated(keep='first')]

In [16]:
# Let's see the shape so far
df.shape

(103, 3)

In [17]:
df.head()

,PostalCode,Borough,Neighborhood
PostalCode,,,
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,M7A,Queen's Park,Queen's Park


Next let's reset the index back and drop the current index

In [18]:
df.reset_index(drop=True, inplace=True)

# Let's see the first few rows
df.head(11)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [19]:
df.shape

(103, 3)

### Appending the Latitude and Longitude data 

Let's define a simple method that we can apply to each Borough to get its Latitude and Longitude using the apply() method

In [0]:
def latitude_longitude(Borough):
    """ Method takes a Series object and returns
    a list of Latitude and corresponding Longitude data,
    using the geopy library.
    This method also prints out the coordinate data"""
    
    address = Borough
    
    geolocator = Nominatim(user_agent="CA_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
    return [latitude, longitude]


Let's append the list containing corresponding lat and lon data to column Latitude

In [21]:
df['Latitude'] = df.Borough.apply(latitude_longitude)

The geograpical coordinate of North York are 43.7708175, -79.4132998.
The geograpical coordinate of North York are 43.7708175, -79.4132998.
The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.
The geograpical coordinate of North York are 43.7708175, -79.4132998.
The geograpical coordinate of Queen's Park are 43.6599803, -79.3903686.
The geograpical coordinate of Etobicoke are 43.67145915, -79.5524920661167.
The geograpical coordinate of Scarborough are 54.2847601, -0.4090339.
The geograpical coordinate of North York are 43.7708175, -79.4132998.
The geograpical coordinate of East York are 43.6913391, -79.3278212.
The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.
The geograpical coordinate of North York are 43.7708175, -79.4132998.
The geograpical coordinate of Etobicoke are 43.67145915, -79.5524920661167.
The geograpical coordinate of Scarborough are 54.2847601, -0.4090339.
The geograpical coordinate of North York are 43.77081

In [22]:
# Lets see the updated data with Latitude containing lists of lats and lons data

df.head()

,PostalCode,Borough,Neighborhood,Latitude
0,M3A,North York,Parkwoods,"[43.7708175, -79.4132998]"
1,M4A,North York,Victoria Village,"[43.7708175, -79.4132998]"
2,M5A,Downtown Toronto,"Harbourfront, Regent Park","[43.6541737, -79.3808116451341]"
3,M6A,North York,"Lawrence Heights, Lawrence Manor","[43.7708175, -79.4132998]"
4,M7A,Queen's Park,Queen's Park,"[43.6599803, -79.3903686]"


Next let's loop through the data frame and separate Latitude from Longitude and make lat and lons just numbers not lists

In [0]:
lon_list = []
for i, j in df.iterrows():
    lon_list.append(j.Latitude[1])
    j.Latitude = j.Latitude[0]
    
# next let's assign the lon_list as the value of the Longitude Column

df['Longitude'] = lon_list

In [24]:
# let's view the changes

df.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7708,-79.413300
1,M4A,North York,Victoria Village,43.7708,-79.413300
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542,-79.380812
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7708,-79.413300
4,M7A,Queen's Park,Queen's Park,43.66,-79.390369
5,M9A,Etobicoke,Islington Avenue,43.6715,-79.552492
6,M1B,Scarborough,"Rouge, Malvern",54.2848,-0.409034
7,M3B,North York,Don Mills North,43.7708,-79.413300
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.6913,-79.327821
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6542,-79.380812


In [25]:
# Let's print the shape of the data frame

df.shape

(103, 5)

### EDA of Toronto Neighborhood Clusters

Let's explore and cluster the neighborhoods in Toronto. We can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did earlier to the New York City data.

In [26]:
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7708,-79.413300
1,M4A,North York,Victoria Village,43.7708,-79.413300
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542,-79.380812
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7708,-79.413300
4,M7A,Queen's Park,Queen's Park,43.66,-79.390369
5,M9A,Etobicoke,Islington Avenue,43.6715,-79.552492
6,M1B,Scarborough,"Rouge, Malvern",54.2848,-0.409034
7,M3B,North York,Don Mills North,43.7708,-79.413300
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.6913,-79.327821
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6542,-79.380812


Let's see the unique Borough names we have in the Data Frame

In [27]:
df.Borough.unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

Let's loop through the data frame and get the index of Boroughs that end with 'Toronto'

In [0]:
toronto_list = [i for i in df.index if df.iloc[i,1].endswith('Toronto')]

Let's slice through the data frame to select only these rows of Boroughs ending with Toronto

In [0]:
Toronto_df = df.iloc[toronto_list,].reset_index(drop=True)

let's visualize the first 10 rows

In [30]:
Toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542,-79.380812
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.6542,-79.380812
2,M5C,Downtown Toronto,St. James Town,43.6542,-79.380812
3,M4E,East Toronto,The Beaches,43.654,-79.387207
4,M5E,Downtown Toronto,Berczy Park,43.6542,-79.380812
5,M5G,Downtown Toronto,Central Bay Street,43.6542,-79.380812
6,M6G,Downtown Toronto,Christie,43.6542,-79.380812
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.6542,-79.380812
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.654,-79.387207
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.6542,-79.380812


Lets see the unique Boroughs once again in Toronto

In [31]:
Toronto_df.Borough.unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

Let's get the geographic coordinates of Toronto

In [32]:
address = 'Toronto'

geolocator = Nominatim(user_agent='CA_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


let's visualize Toronto and the neighborhoods in it.

In [33]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = "{}, {}".format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto  # Feel free to zoom in to see more

### Let's explore the data using foursquare API

In [34]:
CLIENT_ID = 'L2XUYCRD44VP2YBDMZ3XY0Z4XKO1JODM5SMG3IIYRZ5330FD' # your Foursquare ID
CLIENT_SECRET = '5KI00FG4RT2VJQTIDPXFY1N4AOKTIXLIZEAO0AHPP3DPD0U1' # your Foursquare Secret
VERSION = '20190727' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L2XUYCRD44VP2YBDMZ3XY0Z4XKO1JODM5SMG3IIYRZ5330FD
CLIENT_SECRET:5KI00FG4RT2VJQTIDPXFY1N4AOKTIXLIZEAO0AHPP3DPD0U1


Let's confirm the shape of Toronto_df

In [35]:
Toronto_df.shape  # This shows there are 38 rows and 5 columns

(38, 5)

Next let's see the unique borough names in Toronto_df

In [36]:
Toronto_df.Borough.unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

### Okay so lets explore the first location in 'Downtown Toronto'

Let's get the details of the first entry 

In [37]:
first_entry_Downtown_Toronto = Toronto_df[Toronto_df.Borough =='Downtown Toronto'].head(1)

first_entry_Downtown_Toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542,-79.380812


Now, let's get the top 100 venues that are in the first entry within a radius of 500 meters.

First let's create the get request url

In [0]:
radius = 500
LIMIT=100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_entry_Downtown_Toronto.Latitude[0], 
    first_entry_Downtown_Toronto.Longitude[0], 
    radius, 
    LIMIT)

Send the GET request and examine the resutls

In [39]:
result = requests.get(url).json()

result.keys()

dict_keys(['meta', 'response'])

In [40]:
result

{'meta': {'code': 200, 'requestId': '5d3cf0305fc8ed00314dc1a9'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-57eda381498ebe0e6ef40972-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d103951735',
         'name': 'Clothing Store',
         'pluralName': 'Clothing Stores',
         'primary': True,
         'shortName': 'Apparel'}],
       'id': '57eda381498ebe0e6ef40972',
       'location': {'address': '220 Yonge St',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Dundas St W',
        'distance': 193,
        'formattedAddress': ['220 Yonge St (at Dundas St W)',
         'Toronto ON M5B 2H1',
         'Canada'],
        'labeledLatLngs':

function that extracts the category of the venue

In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [42]:
venues = result['response']['groups'][0]['items']

venues

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-0-57eda381498ebe0e6ef40972-0',
  'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d103951735',
     'name': 'Clothing Store',
     'pluralName': 'Clothing Stores',
     'primary': True,
     'shortName': 'Apparel'}],
   'id': '57eda381498ebe0e6ef40972',
   'location': {'address': '220 Yonge St',
    'cc': 'CA',
    'city': 'Toronto',
    'country': 'Canada',
    'crossStreet': 'at Dundas St W',
    'distance': 193,
    'formattedAddress': ['220 Yonge St (at Dundas St W)',
     'Toronto ON M5B 2H1',
     'Canada'],
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.65591027779457,
      'lng': -79.38064099181345}],
    'lat': 43.65591027779457,
    'lng': -79.38064099181345,
    'neighborhood': 'Downtown Toronto

In [0]:
nearby_venues = json_normalize(venues) # flatten JSON

In [44]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,distance
0,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641,193
1,Elgin And Winter Garden Theatres,Theater,43.653394,-79.378507,204
2,Indigo,Bookstore,43.653515,-79.380696,73
3,Ed Mirvish Theatre,Theater,43.655102,-79.379768,133
4,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980,148


And how many venues were returned by Foursquare?

In [45]:
print('The total number of venues returned is {}'.format(len(nearby_venues)))

The total number of venues returned is 100


### Okay so lets explore venues in Central Toronto'

In [46]:
central_toronto_df = Toronto_df[Toronto_df.Borough == 'Central Toronto']

central_toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
17,M4N,Central Toronto,Lawrence Park,43.654,-79.387207
18,M5N,Central Toronto,Roselawn,43.654,-79.387207
19,M4P,Central Toronto,Davisville North,43.654,-79.387207
20,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.654,-79.387207
22,M4R,Central Toronto,North Toronto West,43.654,-79.387207
23,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.654,-79.387207
25,M4S,Central Toronto,Davisville,43.654,-79.387207
28,M4T,Central Toronto,"Moore Park, Summerhill East",43.654,-79.387207
30,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.654,-79.387207


Let's create a function to repeat the same process to all the neighborhoods in Central Toronto

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['location']['distance']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue Distance']
    
    return(nearby_venues)

In [48]:
central_toronto_venues = getNearbyVenues(names=central_toronto_df['Neighborhood'],
                                   latitudes=central_toronto_df['Latitude'],
                                   longitudes=central_toronto_df['Longitude']
                                  )


Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
North Toronto West
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West


Let's check the size of the resulting dataframe


In [49]:
print(central_toronto_venues.shape)

central_toronto_venues.head()

(693, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Distance
0,Lawrence Park,43.653963,-79.387207,Downtown Toronto,43.653232,-79.385296,Neighborhood,174
1,Lawrence Park,43.653963,-79.387207,Japango,43.655268,-79.385165,Sushi Restaurant,219
2,Lawrence Park,43.653963,-79.387207,Cafe Plenty,43.654571,-79.389450,Café,192
3,Lawrence Park,43.653963,-79.387207,Textile Museum of Canada,43.654396,-79.386500,Art Museum,74
4,Lawrence Park,43.653963,-79.387207,Poke Guys,43.654895,-79.385052,Poke Place,202


Let's check how many venues were returned per neighborhood

In [50]:
central_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Distance
Neighborhood,,,,,,,
Davisville,77,77,77,77,77,77,77
Davisville North,77,77,77,77,77,77,77
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",77,77,77,77,77,77,77
"Forest Hill North, Forest Hill West",77,77,77,77,77,77,77
Lawrence Park,77,77,77,77,77,77,77
"Moore Park, Summerhill East",77,77,77,77,77,77,77
North Toronto West,77,77,77,77,77,77,77
Roselawn,77,77,77,77,77,77,77
"The Annex, North Midtown, Yorkville",77,77,77,77,77,77,77


Let's find out how many unique categories can be curated from all the returned venues

In [51]:
print('There are {} unique categories of Venues'.format(central_toronto_venues['Venue Category'].nunique()))

There are 55 unique categories of Venues


### Analyze Each Neighborhood

In [0]:
# one hot encoding
central_toronto_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [0]:
# add neighborhood column back to dataframe
central_toronto_onehot['Neighborhood'] = central_toronto_venues['Neighborhood'] 

In [54]:
# move neighborhood column to the first column
fixed_columns = [central_toronto_onehot.columns[-1]] + list(central_toronto_onehot.columns[:-1])
central_toronto_onehot = central_toronto_onehot[fixed_columns]

pd.set_option('display.max_columns', None)
central_toronto_onehot.head()

,Vegetarian / Vegan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bar,Breakfast Spot,Bubble Tea Shop,Café,Chinese Restaurant,Coffee Shop,Concert Hall,Cosmetics Shop,Costume Shop,Dance Studio,Dessert Shop,Donut Shop,Food Court,French Restaurant,Gastropub,Gift Shop,Greek Restaurant,Gym,Hotel,Ice Cream Shop,Indian Restaurant,Japanese Restaurant,Jazz Club,Korean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Neighborhood,Office,Opera House,Park,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Seafood Restaurant,Smoke Shop,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,University
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Let's examine the new Data frame size

In [55]:
central_toronto_onehot.shape

(693, 55)

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [56]:
central_toronto_grouped = central_toronto_onehot.groupby('Neighborhood').mean().reset_index()
central_toronto_grouped

,Neighborhood,Vegetarian / Vegan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bar,Breakfast Spot,Bubble Tea Shop,Café,Chinese Restaurant,Coffee Shop,Concert Hall,Cosmetics Shop,Costume Shop,Dance Studio,Dessert Shop,Donut Shop,Food Court,French Restaurant,Gastropub,Gift Shop,Greek Restaurant,Gym,Hotel,Ice Cream Shop,Indian Restaurant,Japanese Restaurant,Jazz Club,Korean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Office,Opera House,Park,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Seafood Restaurant,Smoke Shop,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,University
0,Davisville,0.012987,0.012987,0.038961,0.012987,0.012987,0.012987,0.025974,0.038961,0.025974,0.051948,0.025974,0.077922,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.038961,0.012987,0.025974,0.012987,0.012987,0.012987
1,Davisville North,0.012987,0.012987,0.038961,0.012987,0.012987,0.012987,0.025974,0.038961,0.025974,0.051948,0.025974,0.077922,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.038961,0.012987,0.025974,0.012987,0.012987,0.012987
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.012987,0.012987,0.038961,0.012987,0.012987,0.012987,0.025974,0.038961,0.025974,0.051948,0.025974,0.077922,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.038961,0.012987,0.025974,0.012987,0.012987,0.012987
3,"Forest Hill North, Forest Hill West",0.012987,0.012987,0.038961,0.012987,0.012987,0.012987,0.025974,0.038961,0.025974,0.051948,0.025974,0.077922,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.038961,0.012987,0.025974,0.012987,0.012987,0.012987
4,Lawrence Park,0.012987,0.012987,0.038961,0.012987,0.012987,0.012987,0.025974,0.038961,0.025974,0.051948,0.025974,0.077922,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.038961,0.012987,0.025974,0.012987,0.012987,0.012987
5,"Moore Park, Summerhill East",0.012987,0.012987,0.038961,0.012987,0.012987,0.012987,0.025974,0.038961,0.025974,0.051948,0.025974,0.077922,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.025974,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.025974,0.012987,0.012987,0.012987,0.012987,0.038961,0.012987,0.025974,0.012987,0.012987,0.012987
6,North Toronto West,0.012987,0.012987,0.038961,0.012987,0.012987,0.012987,0.025974,0.038961,0.025974,0.051948,0.025974,0.077922,0.012987,0.01

In [57]:
central_toronto_grouped.shape

(9, 55)

Let's print each neighborhood along with the top 5 most common venues

In [58]:
num_top_venues = 5

for hood in central_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = central_toronto_grouped[central_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2       Art Gallery  0.04
3  Sushi Restaurant  0.04
4    Breakfast Spot  0.04


----Davisville North----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2       Art Gallery  0.04
3  Sushi Restaurant  0.04
4    Breakfast Spot  0.04


----Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2       Art Gallery  0.04
3  Sushi Restaurant  0.04
4    Breakfast Spot  0.04


----Forest Hill North, Forest Hill West----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2       Art Gallery  0.04
3  Sushi Restaurant  0.04
4    Breakfast Spot  0.04


----Lawrence Park----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2       Art Gallery  0.04
3  Sushi Restaurant  0.04
4    Breakfast Spot  0.04


----Moore Park, Summerhill

Lets put this into a Pandas Data frame

first let's write a function tp sort the venues in descending order

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Next let's create the new Data frame and display the Top 10 venues for each neighborhood

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = central_toronto_grouped['Neighborhood']

for ind in np.arange(central_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
1,Davisville North,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
3,"Forest Hill North, Forest Hill West",Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
4,Lawrence Park,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop


### Cluster Neighborhoods

In [0]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

Run k-means to cluster the neighborhood into 5 clusters.

In [75]:
# set number of clusters
kclusters = 5

c_toronto_grouped_clustering = central_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(c_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:969: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [63]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

c_toronto_merged = central_toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
c_toronto_merged = c_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

c_toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M4N,Central Toronto,Lawrence Park,43.654,-79.387207,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
18,M5N,Central Toronto,Roselawn,43.654,-79.387207,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
19,M4P,Central Toronto,Davisville North,43.654,-79.387207,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
20,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.654,-79.387207,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
22,M4R,Central Toronto,North Toronto West,43.654,-79.387207,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop


Finally, let's visualize the resulting clusters

In [76]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(c_toronto_merged['Latitude'], c_toronto_merged['Longitude'], c_toronto_merged['Neighborhood'], c_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.



__Cluster 1__

In [68]:
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 0, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
18,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
19,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
20,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
22,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
23,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
25,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
28,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
30,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop


__Cluster Two__

In [69]:
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 1, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


__Cluster Three__

In [70]:
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 2, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


__Cluster Four__

In [71]:
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 3, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 0, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]

In [73]:
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 4, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
18,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
19,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
20,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
22,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
23,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
25,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
28,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
30,Central Toronto,0,Coffee Shop,Café,Art Gallery,Sushi Restaurant,Breakfast Spot,Restaurant,Ice Cream Shop,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop
